#RO Project#

1. Read instances' data from file
2. Initialize the parameters read from file
3. n = number of customers
4. c = matrix of costs
5. ml = max load of the vehicle
6. l = linehaul values vector
7. b = backhaul values vector
8. Initialize the matrix s of savings 
9. for i = 1:n
    1. for j = 1:n
        1. If i == j
            1. s_ij = +inf
        2. else
            1. s_ij = c_i0 + c_0j - c_ij
10. Routes = list of empty routes
11. for i = 1:n
    1. Routes_i = (0,i,0)
12. Order the savings s in a non increasing fashion
13. for each route in Routes:
    1. while this route can be feasibly (respecting the constraints) merged further:
	   1.determine the first saving s_ki or s_jl in the ordered list that can feasibly be used to merge the current route with another route containing either the arc (k, 0) or (0, l). 
       	   2. merge the two routes.



In [1]:
import numpy as np
import sys

In [2]:
filename = 'Instances/A1.txt'
with open(filename, 'r') as f:
    content = f.read()
    print(content)

25
1
8
12000   16000   0   1550
9795   21510   0   549   0
7758   10895   0   558   0
12927   2464   0   851   0
23286   5538   0   388   0
7416   15826   0   194   0
21524   24879   483   0   0
22337   17915   389   0   0
7642   9722   435   0   0
3707   20036   452   0   0
15492   2721   343   0   0
12430   10638   444   0   0
13595   1918   350   0   0
4290   2721   307   0   0
18684   5268   810   0   0
8686   9546   1002   0   0
23140   10989   476   0   0
5263   14289   674   0   0
19557   17564   691   0   0
23281   29684   675   0   0
374   22683   410   0   0
18804   9082   351   0   0
5444   25004   786   0   0
20390   25745   59   0   0
4310   21692   362   0   0
8526   2495   550   0   0



In [3]:
content = content.split('\n')[0:-1] #-1 because the last element is a newline, so we skip it

In [4]:
n_of_customers = int(content[0])
n_of_vehicles = int(content[2])
capacity = int(content[3].split()[3])
linehaul_vector = [0] + [int(i.split()[2]) for i in content[4:]] #linehaul data is in the fourth column
backhaul_vector = [0] + [int(i.split()[3]) for i in content[4:]] #and backhaul is in the third

In [5]:
import math

def euclidean_distance2D(point1, point2):
    
    x1, y1 = point1
    x2, y2 = point2
    
    return (math.sqrt((x1-x2)**2 + (y1-y2)**2))

In [6]:
cost_matrix = []

x_coords = [int(i.split()[0]) for i in content[3:]]
y_coords = [int(i.split()[1]) for i in content[3:]]

for i, (x1, y1) in enumerate(zip(x_coords, y_coords)): #could also enumerate over y_coords, the dimensions are equal
    
    current_row = []
    
    for j, (x2, y2) in enumerate(zip(x_coords, y_coords)):
        
        current_row += [euclidean_distance2D((x1,y1), (x2,y2))]
        
    cost_matrix += [current_row]
        
        

In [7]:
cost_matrix = np.array(cost_matrix)
savings_matrix = np.zeros((n_of_customers + 1, n_of_customers + 1))

In [8]:
for i in range(n_of_customers + 1):
    for j in range(n_of_customers + 1):
        if (i==j):
            savings_matrix[i][j] = sys.maxsize
        else:
            savings_matrix[i][j] = cost_matrix[i][0] + cost_matrix[0][j] - cost_matrix[i][j]

In [9]:
routes = []

In [10]:
for i in range(n_of_customers):
    
    routes += [(0, i+1, 0)]

In [11]:
ordered_savings = []

for (i, j), el in np.ndenumerate(savings_matrix):
    ordered_savings += [(el, (i, j))]

ordered_savings.sort(key=lambda x: x[0], reverse=True)
ordered_i_savings = list(ordered_savings)
ordered_j_savings = list(ordered_savings)
ordered_i_savings.sort(key=lambda x: (x[1][0], -x[0]))
ordered_j_savings.sort(key=lambda x: (x[1][1], -x[0]))


In [12]:
def is_route_valid (route_to_check):
    
    is_route_possible = True
    cost_of_possible_route_linehaul = 0
    cost_of_possible_route_backhaul = 0
    i = 0
    
    for node_touched in route_to_check[1:-1]:
        
        if linehaul_vector[node_touched] == 0:#linehaul_vector is 0 if a node is not a linehaul node, so we have to go to the next for
            break
        else:
            cost_of_possible_route_linehaul += linehaul_vector[node_touched]
            i+=1
            
    starting_point = i
    if i < len(route_to_check[1:-1]):
        for node_touched in route_to_check[starting_point+1:-1]:
            
            if backhaul_vector[node_touched] == 0 and i != len(route_to_check): #we are trying to linehaul AFTER we started backhauling... constraint broken:
                is_route_possible = False
                #print(i, backhaul_vector[node_touched], node_touched)
            else:
                cost_of_possible_route_backhaul += backhaul_vector[node_touched]
                i+=1
                
        
    if (cost_of_possible_route_backhaul > capacity or cost_of_possible_route_linehaul > capacity):
        is_route_possible = False
        
    if (cost_of_possible_route_linehaul == 0): # we are only backhauling
        is_route_possible = False
        
        
    return is_route_possible

    

In [13]:
def is_route_merge_valid(id_route1, id_route2):
    
    """First, try if you can merge it with the end of route1 to end of route2"""
        
    possible_route = list( routes[id_route1][:-1] + routes[id_route2][1:]) #the merge itself is just skipping the left and right
    
    is_right_possible = is_route_valid(possible_route)
    """Now the other possible route."""
        
    possible_route = list( routes[id_route2][:-1] + routes[id_route1][1:]) #the merge itself is just skipping the left and right
    
    is_left_possible = is_route_valid(possible_route)
    
    #print(is_left_possible, is_right_possible)
        
    if not is_left_possible and not is_right_possible:
        return ('')
    elif is_left_possible and is_right_possible:
        return ('lr')
    elif is_left_possible:
        return 'l'
    else:
        return 'r'
        
                


In [14]:
def find_first_merge(id_curr_route, available_merges):
    
    
    for saving in ordered_i_savings:
        
        if saving[1][0] > id_curr_route+1:
            break
        
        if(saving[1][0] == id_curr_route+1 and saving[0] != sys.maxsize):
            
            for merge in available_merges:
                if saving[1][1] == merge[0]+1:
                    return merge
        
    for saving in ordered_j_savings:
        
        if saving[1][0] > id_curr_route+1:
            break
            
        if(saving[1][0] == id_curr_route and saving[0] != sys.maxsize):
            
            for merge in available_merges:

                if saving[1][1] == merge[0]+1:
                    return merge
    

In [15]:
def merge_two_routes(id_route, found_id, direction_of_merge):

    if id_route== found_id:
        return
    
    tmp_list1 = list(routes[id_route])
    tmp_list2 = list(routes[found_id])
        
    tmp_list1.pop(0)
    tmp_list1.pop(-1)
    tmp_list2.pop(0)
    tmp_list2.pop(-1)
    
    if direction_of_merge == 'l':
        
        routes[id_route] = tuple([0]+tmp_list2+tmp_list1+[0])
    
    if direction_of_merge == 'r':
        
        routes[id_route] = tuple([0]+tmp_list1+tmp_list2+[0])
    
    routes[found_id] = (-1,-1,-1)


    

In [16]:
direction_of_merge = is_route_merge_valid(10, 18)
print(direction_of_merge)

lr


In [17]:
for id_route1, route in enumerate(routes):
    
    while True:
                
        if (routes[id_route1] == (-1,-1,-1)):
            break
        
        """
        Next thing to do should be modify the for underneath this comment so that it saves an array of valid routes, and at the end, if the array is not empty,
        check which one of the possible merges is the best and then merge with that
        
        """
                
        found_id = -1
        direction_of_merge = 'l'
        
        available_merges = []
        for id_route2, route2 in enumerate(routes):

            if (routes[id_route2] == (-1,-1,-1)):
                continue
               
            if (id_route1 != id_route2):
                    direction_of_merge = is_route_merge_valid(id_route1, id_route2)
                    if direction_of_merge != "":
                            if direction_of_merge == 'l' or direction_of_merge == 'r':
                                available_merges += [(id_route2, direction_of_merge)]
                            else:
                                available_merges += [(id_route2, 'l')]
                                available_merges += [(id_route2, 'r')]
                    
        
        if (len(available_merges) == 0):
            break
        #print(available_merges)
        found_id, direction_of_merge= find_first_merge(id_route1, available_merges)
        if routes[found_id] != (-1,-1,-1):
            merge_two_routes(id_route1, found_id, direction_of_merge)
                 

In [18]:
routes

[(0, 13, 23, 24, 22, 1, 5, 2, 0),
 (-1, -1, -1),
 (0, 25, 10, 12, 3, 4, 0),
 (-1, -1, -1),
 (-1, -1, -1),
 (0, 7, 19, 6, 0),
 (-1, -1, -1),
 (0, 15, 8, 0),
 (0, 17, 20, 9, 0),
 (-1, -1, -1),
 (0, 14, 11, 0),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (0, 18, 21, 16, 0),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1)]

In [19]:
for route in routes:
    if route == (-1,-1,-1):
        continue
    total_l = 0
    total_b = 0
    for el in route:
        if el == 0:
            continue
        
        total_l += linehaul_vector[el]
        total_b += backhaul_vector[el]
    print(route)
    print(total_l)
    print(total_b)

(0, 13, 23, 24, 22, 1, 5, 2, 0)
1514
1301
(0, 25, 10, 12, 3, 4, 0)
1243
1239
(0, 7, 19, 6, 0)
1547
0
(0, 15, 8, 0)
1437
0
(0, 17, 20, 9, 0)
1536
0
(0, 14, 11, 0)
1254
0
(0, 18, 21, 16, 0)
1518
0


In [20]:
for route in routes:
    
    
    if route != (-1, -1, -1):
        
        print ('Route:')
        print (route)
        
        for item in route[1:-1]:
            
            if linehaul_vector[item] != 0:
                print('L')
                
            if backhaul_vector[item] != 0:
                print('B')
                


Route:
(0, 13, 23, 24, 22, 1, 5, 2, 0)
L
L
L
L
B
B
B
Route:
(0, 25, 10, 12, 3, 4, 0)
L
L
L
B
B
Route:
(0, 7, 19, 6, 0)
L
L
L
Route:
(0, 15, 8, 0)
L
L
Route:
(0, 17, 20, 9, 0)
L
L
L
Route:
(0, 14, 11, 0)
L
L
Route:
(0, 18, 21, 16, 0)
L
L
L


In [21]:
total_cost = 0

for route in routes:
    
    if route != (-1, -1, -1):
        
        for idx, item in enumerate(route[:-1]):
            
            total_cost+=cost_matrix[item, route[idx+1]]

In [22]:
total_cost

282894.4978628099